In [ ]:
!git clone https://github.com/NonniHlolli/cycleNew.git

In [ ]:
import sys
sys.path.append('/cycleNew')
!pip install visdom

In [ ]:
! npm install -g localtunnel
get_ipython().system_raw('python3 -m pip install visdom')
get_ipython().system_raw('python3 -m visdom.server -port 8097 >> visdomlog.txt 2>&1 &')   
get_ipython().system_raw('lt --port 8097 >> url.txt 2>&1 &')   
import visdom  

In [ ]:
import itertools
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from PIL import Image
import torch
from utilities import *
from data import *
from options import *
from gener import *
from discr import *
from residBlock import *
from imgBuffer import *

# INITIALIZATION

# Set training options
opt = Options(decay_epoch=10,dataroot = 'apple2orange/', n_cpu = 4, cuda = True, n_epochs=50 )


# Initialize the Networks
netG_A2B = Generator()
netG_B2A = Generator()
netD_A = Discriminator()
netD_B = Discriminator()

# Settings for GPU usage
if opt.cuda:
    netG_A2B.cuda()
    netG_B2A.cuda()
    netD_A.cuda()
    netD_B.cuda()
    Tensor = torch.cuda.FloatTensor
else:
    Tensor = torch.Tensor

# Initialize weights
netG_A2B.apply(weights_init_normal)
netG_B2A.apply(weights_init_normal)
netD_A.apply(weights_init_normal)
netD_B.apply(weights_init_normal)

# Set the three losses
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

# Initialize Adam optimizers for the networks
optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Initialize the learning rate scheduler
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)

# allocate memory
input_A = Tensor(1, 3, 256, 256)
input_B = Tensor(1, 3, 256, 256)
target_real = Variable(Tensor(1).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(1).fill_(0.0), requires_grad=False)

# Create the image buffer
fake_A_buffer = ImageBuffer()
fake_B_buffer = ImageBuffer()

# Load in the training dataset
transforms_ =[  transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
dataloader = DataLoader(ImageDataset(opt.dataroot, transforms_=transforms_, unaligned=True), 
                        batch_size=.1, shuffle=True, num_workers=opt.n_cpu)

# Start the logger
logger = Logger(opt.n_epochs, len(dataloader))



In [ ]:
# TRAINING

for epoch in range(opt.epoch, opt.n_epochs):
    for i, batch in enumerate(dataloader):
        
        # get real images
        real_A = Variable(input_A.copy_(batch['A']))
        real_B = Variable(input_B.copy_(batch['B']))
        
        # STEP FOR GENERATORS
        optimizer_G.zero_grad()

        # Identity loss term 
        # L1 loss on difference on transformed input and input
        same_B = netG_A2B(real_B)
        loss_identity_B = criterion_identity(same_B, real_B)*5.0
        same_A = netG_B2A(real_A)
        loss_identity_A = criterion_identity(same_A, real_A)*5.0

        # GAN loss term
        # MSE loss on correct discriminator classification
        fake_B = netG_A2B(real_A)
        pred_fake = netD_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake, target_real)
        fake_A = netG_B2A(real_B)
        pred_fake = netD_A(fake_A)
        loss_GAN_B2A = criterion_GAN(pred_fake, target_real)
        

        # Cycle loss term
        # L1 loss on difference on cycle translated input and input
        recovered_A = netG_B2A(fake_B)
        loss_cycle_ABA = criterion_cycle(recovered_A, real_A)*10.0
        recovered_B = netG_A2B(fake_A)
        loss_cycle_BAB = criterion_cycle(recovered_B, real_B)*10.0

        # Total loss
        loss_G = loss_identity_A + loss_identity_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB
        loss_G.backward()
        
        optimizer_G.step()

        # STEP FOR DISCRIMINATORS
        optimizer_D_A.zero_grad()
        optimizer_D_B.zero_grad()

        # loss for misclassifying the real image
        pred_real = netD_A(real_A)
        loss_D_real = criterion_GAN(pred_real, target_real)
        
        pred_real = netD_B(real_B)
        loss_D_real = criterion_GAN(pred_real, target_real)

        # loss for misclassifying the fake image from the image buffer
        fake_A = fake_A_buffer.push_and_pop(fake_A)
        pred_fake = netD_A(fake_A.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)
        
        fake_B = fake_B_buffer.push_and_pop(fake_B)
        pred_fake = netD_B(fake_B.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        # Sum of the losses
        loss_D_A = (loss_D_real + loss_D_fake)*0.5
        loss_D_A.backward()
        
        loss_D_B = (loss_D_real + loss_D_fake)*0.5
        loss_D_B.backward()

        optimizer_D_A.step()
        optimizer_D_B.step()
        
        # Send the losses to the logger
        losses = {'loss_G': loss_G.item(), 'loss_G_identity': (loss_identity_A.item() + loss_identity_B.item()), 'loss_G_GAN': (loss_GAN_A2B.item() + loss_GAN_B2A.item()),
                    'loss_G_cycle': (loss_cycle_ABA.item() + loss_cycle_BAB.item()), 'loss_D': (loss_D_A.item() + loss_D_B.item())}
        logger.log(losses)

    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

# Save models at the end
torch.save(netG_A2B.state_dict(), 'output/netG_A2B.pth')
torch.save(netG_B2A.state_dict(), 'output/netG_B2A.pth')
torch.save(netD_A.state_dict(), 'output/netD_A.pth')
torch.save(netD_B.state_dict(), 'output/netD_B.pth')

In [ ]:
import argparse
import sys
import os

import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch

# INITIALIZE FOR TESTING

isCuda = True
# Initialize generators
netG_A2B = Generator(3,3)
netG_B2A = Generator(3,3)

if isCuda:
    netG_A2B.cuda()
    netG_B2A.cuda()
    Tensor = torch.cuda.FloatTensor
else:
    Tensor = torch.Tensor

# Load the models
netG_A2B.load_state_dict(torch.load('output/netG_A2B.pth'))
netG_B2A.load_state_dict(torch.load('output/netG_B2A.pth'))

# Set models to test mode
netG_A2B.eval()
netG_B2A.eval()

# allocate memory
input_A = Tensor(1,3,256,256)
input_B = Tensor(1,3,256,256)

# Load in the testing dataset
transforms_ = [ transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset('apple2orange/', transforms_=transforms_, mode='test'), 
                        batch_size=1, shuffle=False, num_workers=4)

# TEST
for i, batch in enumerate(dataloader):
    # Get the real test images
    real_A = Variable(input_A.copy_(batch['A']))
    real_B = Variable(input_B.copy_(batch['B']))

    # Translate images
    fake_B = 0.5*(netG_A2B(real_A).data + 1.0)
    fake_A = 0.5*(netG_B2A(real_B).data + 1.0)

    # Save the images
    save_image(fake_A, 'output/A/%04d.png' % (i+1))
    save_image(fake_B, 'output/B/%04d.png' % (i+1))
